# 🎓 CSC582: Relational to Key-Value Database Migration
## E-Commerce Database → Redis Cluster

**Course:** CSC582 - Data Warehouse and Mining Systems

This notebook demonstrates:
- Relational database design (SQLite)
- Mapping strategy to Key-Value model
- Redis Cluster with sharding & replication
- Key-Value operations (SET, GET, HSET, HGET, etc.)

---

## 📦 Step 1: Install Redis Server

In [ ]:
# Install Redis Server and Python client
!apt-get update -qq
!apt-get install -y redis-server redis-tools > /dev/null
!pip install redis -q

print("✅ Redis installed successfully!")
!redis-server --version

## 🚀 Step 2: Start Redis Server

In [ ]:
import subprocess
import time

# Start Redis server in background
subprocess.Popen(['redis-server', '--daemonize', 'yes'])
time.sleep(2)

# Verify Redis is running
result = subprocess.run(['redis-cli', 'ping'], capture_output=True, text=True)
if result.stdout.strip() == 'PONG':
    print("✅ Redis server is running!")
else:
    print("❌ Redis failed to start")

---
# 📊 Part 1: Relational Database

We create an E-Commerce database with 3 tables:
- **Customers**: Customer information
- **Products**: Product catalog
- **Orders**: Customer orders

In [ ]:
import sqlite3
import pandas as pd

# Create SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript("""
    CREATE TABLE customers (
        customer_id     INTEGER PRIMARY KEY,
        first_name      TEXT NOT NULL,
        last_name       TEXT NOT NULL,
        email           TEXT UNIQUE NOT NULL,
        phone           TEXT,
        city            TEXT,
        country         TEXT
    );
    
    CREATE TABLE products (
        product_id      INTEGER PRIMARY KEY,
        product_name    TEXT NOT NULL,
        category        TEXT,
        price           REAL NOT NULL,
        stock_quantity  INTEGER DEFAULT 0
    );
    
    CREATE TABLE orders (
        order_id        INTEGER PRIMARY KEY,
        customer_id     INTEGER NOT NULL,
        order_date      TEXT NOT NULL,
        status          TEXT DEFAULT 'pending',
        total_amount    REAL,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
""")

print("✅ Relational schema created!")
print("\n📋 Tables: customers, products, orders")

In [ ]:
# Insert sample data
customers_data = [
    (1, 'Ahmed', 'Al-Rashid', 'ahmed.rashid@email.com', '+966501234567', 'Riyadh', 'Saudi Arabia'),
    (2, 'Fatima', 'Hassan', 'fatima.hassan@email.com', '+966502345678', 'Jeddah', 'Saudi Arabia'),
    (3, 'Mohammed', 'Al-Saud', 'mohammed.saud@email.com', '+966503456789', 'Dammam', 'Saudi Arabia'),
    (4, 'Sara', 'Abdullah', 'sara.abdullah@email.com', '+966504567890', 'Riyadh', 'Saudi Arabia'),
    (5, 'Khalid', 'Omar', 'khalid.omar@email.com', '+966505678901', 'Mecca', 'Saudi Arabia'),
]
cursor.executemany("INSERT INTO customers VALUES (?,?,?,?,?,?,?)", customers_data)

products_data = [
    (101, 'Laptop Pro 15', 'Electronics', 4500.00, 25),
    (102, 'Wireless Mouse', 'Electronics', 150.00, 100),
    (103, 'Office Chair', 'Furniture', 850.00, 30),
    (104, 'Standing Desk', 'Furniture', 2200.00, 15),
    (105, 'Headphones', 'Electronics', 1200.00, 50),
    (106, 'USB-C Hub', 'Electronics', 280.00, 75),
]
cursor.executemany("INSERT INTO products VALUES (?,?,?,?,?)", products_data)

orders_data = [
    (1001, 1, '2024-06-01', 'delivered', 4650.00),
    (1002, 2, '2024-06-05', 'delivered', 3050.00),
    (1003, 1, '2024-06-10', 'shipped', 1200.00),
    (1004, 3, '2024-06-15', 'processing', 2480.00),
    (1005, 4, '2024-06-20', 'pending', 850.00),
    (1006, 5, '2024-06-25', 'delivered', 4780.00),
    (1007, 2, '2024-07-01', 'shipped', 430.00),
]
cursor.executemany("INSERT INTO orders VALUES (?,?,?,?,?)", orders_data)
conn.commit()

print("✅ Sample data inserted!")

In [ ]:
# Display Customers Table
print("👥 CUSTOMERS TABLE")
print("="*70)
df_customers = pd.read_sql_query("SELECT * FROM customers", conn)
display(df_customers)

In [ ]:
# Display Products Table
print("📦 PRODUCTS TABLE")
print("="*70)
df_products = pd.read_sql_query("SELECT * FROM products", conn)
display(df_products)

In [ ]:
# Display Orders Table
print("🛒 ORDERS TABLE")
print("="*70)
df_orders = pd.read_sql_query("SELECT * FROM orders", conn)
display(df_orders)

---
# 🔄 Part 2: Mapping Strategy

## Relational → Key-Value Mapping

| Table | Redis Key Pattern | Type | Example |
|-------|------------------|------|--------|
| customers | `customer:{id}` | HASH | `customer:1` |
| products | `product:{id}` | HASH | `product:101` |
| orders | `order:{id}` | HASH | `order:1001` |

### Secondary Indexes:
- `idx:product:category:{category}` → SET of product IDs
- `idx:customer:{id}:orders` → SET of order IDs
- `idx:orders:status:{status}` → SET of order IDs

In [ ]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Verify connection
print("🔗 Connected to Redis:", r.ping())

---
# 📤 Part 3: Data Migration to Redis

In [ ]:
print("="*60)
print("🚀 MIGRATING DATA TO REDIS")
print("="*60)

# Migrate Customers
print("\n👥 Migrating CUSTOMERS...")
cursor.execute("SELECT * FROM customers")
for row in cursor.fetchall():
    key = f"customer:{row[0]}"
    value = {
        "customer_id": str(row[0]),
        "first_name": row[1],
        "last_name": row[2],
        "email": row[3],
        "phone": row[4] or "",
        "city": row[5] or "",
        "country": row[6] or ""
    }
    r.hset(key, mapping=value)
    print(f"   ✓ HSET {key}")

# Migrate Products
print("\n📦 Migrating PRODUCTS...")
cursor.execute("SELECT * FROM products")
for row in cursor.fetchall():
    key = f"product:{row[0]}"
    value = {
        "product_id": str(row[0]),
        "product_name": row[1],
        "category": row[2],
        "price": str(row[3]),
        "stock_quantity": str(row[4])
    }
    r.hset(key, mapping=value)
    # Add to category index
    r.sadd(f"idx:product:category:{row[2].lower()}", str(row[0]))
    print(f"   ✓ HSET {key}")

# Migrate Orders
print("\n🛒 Migrating ORDERS...")
cursor.execute("SELECT * FROM orders")
for row in cursor.fetchall():
    key = f"order:{row[0]}"
    value = {
        "order_id": str(row[0]),
        "customer_id": str(row[1]),
        "order_date": row[2],
        "status": row[3],
        "total_amount": str(row[4])
    }
    r.hset(key, mapping=value)
    # Add to customer's orders index
    r.sadd(f"idx:customer:{row[1]}:orders", str(row[0]))
    # Add to status index
    r.sadd(f"idx:orders:status:{row[3]}", str(row[0]))
    print(f"   ✓ HSET {key}")

print("\n" + "="*60)
print("✅ MIGRATION COMPLETE!")
print("="*60)

---
# 🔍 Part 4: Key-Value Operations Demo

## 4.1 HGETALL - Get all fields of a hash

In [ ]:
print("📋 Command: HGETALL customer:1")
print("-" * 40)
result = r.hgetall("customer:1")
for field, value in result.items():
    print(f"   {field}: {value}")

## 4.2 HGET - Get specific field

In [ ]:
print("📋 Command: HGET customer:2 email")
print("-" * 40)
email = r.hget("customer:2", "email")
print(f"   Result: {email}")

## 4.3 SMEMBERS - Get set members (Secondary Index)

In [ ]:
print("📋 Command: SMEMBERS idx:product:category:electronics")
print("-" * 40)
electronics = r.smembers("idx:product:category:electronics")
print(f"   Electronics Product IDs: {electronics}")

print("\n📋 Getting product details for each ID:")
for pid in electronics:
    product = r.hgetall(f"product:{pid}")
    print(f"   Product {pid}: {product['product_name']} - ${product['price']}")

## 4.4 Get Customer's Orders

In [ ]:
print("📋 Command: SMEMBERS idx:customer:1:orders")
print("-" * 40)
customer_orders = r.smembers("idx:customer:1:orders")
print(f"   Customer 1's Order IDs: {customer_orders}")

print("\n📋 Order details:")
for oid in customer_orders:
    order = r.hgetall(f"order:{oid}")
    print(f"   Order {oid}: {order['order_date']} - {order['status']} - ${order['total_amount']}")

## 4.5 Get Orders by Status

In [ ]:
print("📋 Command: SMEMBERS idx:orders:status:delivered")
print("-" * 40)
delivered = r.smembers("idx:orders:status:delivered")
print(f"   Delivered Order IDs: {delivered}")

print("\n📋 Command: SMEMBERS idx:orders:status:pending")
print("-" * 40)
pending = r.smembers("idx:orders:status:pending")
print(f"   Pending Order IDs: {pending}")

## 4.6 SET/GET Operations

In [ ]:
print("📋 Command: SET greeting 'Hello, World!'")
r.set("greeting", "Hello, World!")
print("   Result: OK")

print("\n📋 Command: GET greeting")
greeting = r.get("greeting")
print(f"   Result: {greeting}")

## 4.7 List All Keys

In [ ]:
print("📋 Command: KEYS customer:*")
print("-" * 40)
customer_keys = r.keys("customer:*")
print(f"   {customer_keys}")

print("\n📋 Command: KEYS product:*")
print("-" * 40)
product_keys = r.keys("product:*")
print(f"   {product_keys}")

print("\n📋 Command: KEYS order:*")
print("-" * 40)
order_keys = r.keys("order:*")
print(f"   {order_keys}")

---
# 🖥️ Part 5: Redis CLI Commands

Run these commands directly using redis-cli:

In [ ]:
# Run Redis CLI commands directly
print("🖥️ REDIS CLI DEMO")
print("="*60)

!redis-cli HGETALL customer:1

In [ ]:
!redis-cli HGET product:101 product_name

In [ ]:
!redis-cli SMEMBERS idx:product:category:electronics

In [ ]:
!redis-cli SMEMBERS idx:orders:status:delivered

In [ ]:
!redis-cli DBSIZE

In [ ]:
!redis-cli INFO keyspace

---
# 🏗️ Part 6: Redis Cluster Architecture

In a production environment, we would use 8 nodes:

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  MASTER 0   │     │  MASTER 1   │     │  MASTER 2   │     │  MASTER 3   │
│  Port 7000  │     │  Port 7001  │     │  Port 7002  │     │  Port 7003  │
│ Slots 0-4095│     │Slots 4096-  │     │Slots 8192-  │     │Slots 12288- │
│             │     │    8191     │     │   12287     │     │   16383     │
└──────┬──────┘     └──────┬──────┘     └──────┬──────┘     └──────┬──────┘
       │                   │                   │                   │
       ▼                   ▼                   ▼                   ▼
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  REPLICA 0  │     │  REPLICA 1  │     │  REPLICA 2  │     │  REPLICA 3  │
│  Port 7004  │     │  Port 7005  │     │  Port 7006  │     │  Port 7007  │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
```

### Hash-Based Partitioning:
- Total slots: 16384
- Formula: `slot = CRC16(key) mod 16384`

In [ ]:
# Simulate hash slot calculation
import binascii

def crc16(data):
    """Calculate CRC16 (simplified)"""
    crc = 0
    for byte in data.encode():
        crc = ((crc << 8) & 0xFFFF) ^ (byte * 0x1021)
    return crc & 0xFFFF

def get_slot(key):
    return crc16(key) % 16384

def get_node(slot):
    if slot < 4096:
        return "Master 0 (Port 7000)"
    elif slot < 8192:
        return "Master 1 (Port 7001)"
    elif slot < 12288:
        return "Master 2 (Port 7002)"
    else:
        return "Master 3 (Port 7003)"

print("🎯 HASH SLOT DISTRIBUTION")
print("="*60)

keys_to_check = ["customer:1", "customer:2", "product:101", "product:103", "order:1001", "order:1005"]
for key in keys_to_check:
    slot = get_slot(key)
    node = get_node(slot)
    print(f"  Key: {key:15} → Slot: {slot:5} → {node}")

---
# ✅ Summary

This demo covered:

1. ✅ **Relational Model**: 3 tables (Customers, Products, Orders)
2. ✅ **Mapping Strategy**: Row → Hash, with secondary indexes
3. ✅ **Redis Cluster**: 4 Masters + 4 Replicas architecture
4. ✅ **Sharding**: Hash-based partitioning (16384 slots)
5. ✅ **Replication**: Each master has one replica
6. ✅ **Key-Value Operations**: HSET, HGET, HGETALL, SET, GET, SADD, SMEMBERS

---
**Thank you!**